In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',None)
pd.set_option('display.max_row',None)
pd.set_option('display.expand_frame_repr',False)
pd.set_option('display.max_colwidth',None)

In [2]:
deliveries = pd.read_csv('deliveries.csv')
matches = pd.read_csv('matches.csv')

In [3]:
df=deliveries.copy()
mdf=matches.copy()



In [4]:
#df = df.astype({col: 'category' for col in df.select_dtypes(include='object').columns})
#mdf = mdf.astype({col: 'category' for col in df.select_dtypes(include='object').columns})


In [5]:
#mdf.head()

In [6]:
#mdf.tail()

In [7]:
mdf = mdf.rename(columns={'id':'match_id'})

In [8]:
comb=pd.merge(df,mdf,on = 'match_id',how ='left')


In [9]:
comb.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,bowler,non_striker,runs_of_bat,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,1,1,legbyes,0,NaN,NaN,NaN,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,BB McCullum,P Kumar,SC Ganguly,0,1,1,wides,0,NaN,NaN,NaN,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen


In [10]:
def ByInnings(df,current_innings):
    
    df = df[df.innings == current_innings]
    df.reset_index(inplace = True,drop = True)
    
    df['isDot'] = df['runs_of_bat'].apply(lambda x: 1 if x == 0 else 0)
    df['isOne'] = df['runs_of_bat'].apply(lambda x: 1 if x == 1 else 0)
    df['isTwo'] = df['runs_of_bat'].apply(lambda x: 1 if x == 2 else 0)
    df['isThree'] = df['runs_of_bat'].apply(lambda x: 1 if x == 3 else 0)
    df['isFour'] = df['runs_of_bat'].apply(lambda x: 1 if x == 4 else 0)
    df['isSix'] = df['runs_of_bat'].apply(lambda x: 1 if x == 6 else 0)
    
    runs = pd.DataFrame(df.groupby(['batsman'])['runs_of_bat'].sum()).reset_index().rename(columns={'runs_of_bat':'runs'})
    innings = pd.DataFrame(df.groupby(['batsman'])['match_id'].apply(lambda x: len(list(np.unique(x))))).reset_index().rename(columns={'match_id':'innings'})
    balls = pd.DataFrame(df.groupby(['batsman'])['match_id'].count()).reset_index().rename(columns={'match_id':'balls'})
    dismissals = pd.DataFrame(df.groupby(['batsman'])['player_dismissed'].count()).reset_index().rename(columns={'player_dismissed':'dismissals'})
    sixes = pd.DataFrame(df.groupby(['batsman'])['isSix'].sum()).reset_index().rename(columns = {'isSix':'sixes'})
    fours = pd.DataFrame(df.groupby(['batsman'])['isFour'].sum()).reset_index().rename(columns = {'isFour':'fours'})
    
    df = pd.merge(innings,runs,on = 'batsman').merge( balls, on = 'batsman').merge( dismissals, on = 'batsman').merge( fours, on = 'batsman').merge( sixes, on = 'batsman')
    df['RPI'] =df.apply(lambda x: (x['runs']/x['innings']),axis = 1)
    return df


In [11]:
#comb=pd.merge(df,mdf,on = 'match_id',how ='left')


In [12]:
def balls_per_dismissal(balls, dismissals):
    if dismissals > 0:
        return balls/dismissals
    else:
        return balls/1 
    
def balls_per_boundary(balls, boundaries):
    if boundaries > 0:
        return balls/boundaries
    else:
        return balls/1 

In [13]:
def ByCustom(df, current_venue, current_phase, current_opposition):
    # Ensure inputs are lists for consistency
    if not isinstance(current_venue, list):
        current_venue = [current_venue]
    if not isinstance(current_opposition, list):
        current_opposition = [current_opposition]
    
    # Filter by venue (handling single or multiple venues)
    df = df[df.venue.isin(current_venue)]
    
    # Filter by phase
    df = df[df.phase == current_phase]
    
    # Filter by opposition team (handling single or multiple teams)
    df = df[df.bowling_team.isin(current_opposition)]
    
    df.reset_index(inplace=True, drop=True)
    
    df['isDot'] = df['runs_of_bat'].apply(lambda x: 1 if x == 0 else 0)
    df['isOne'] = df['runs_of_bat'].apply(lambda x: 1 if x == 1 else 0)
    df['isTwo'] = df['runs_of_bat'].apply(lambda x: 1 if x == 2 else 0)
    df['isThree'] = df['runs_of_bat'].apply(lambda x: 1 if x == 3 else 0)
    df['isFour'] = df['runs_of_bat'].apply(lambda x: 1 if x == 4 else 0)
    df['isSix'] = df['runs_of_bat'].apply(lambda x: 1 if x == 6 else 0)
    
    runs = pd.DataFrame(df.groupby(['batsman'])['runs_of_bat'].sum()).reset_index().rename(columns={'runs_of_bat':'runs'})
    innings = pd.DataFrame(df.groupby(['batsman'])['match_id'].apply(lambda x: len(list(np.unique(x))))).reset_index().rename(columns={'match_id':'innings'})
    balls = pd.DataFrame(df.groupby(['batsman'])['match_id'].count()).reset_index().rename(columns={'match_id':'balls'})
    dismissals = pd.DataFrame(df.groupby(['batsman'])['player_dismissed'].count()).reset_index().rename(columns={'player_dismissed':'dismissals'})
    sixes = pd.DataFrame(df.groupby(['batsman'])['isSix'].sum()).reset_index().rename(columns = {'isSix':'sixes'})
    fours = pd.DataFrame(df.groupby(['batsman'])['isFour'].sum()).reset_index().rename(columns = {'isFour':'fours'})

    dots = pd.DataFrame(df.groupby(['batsman'])['isDot'].sum()).reset_index().rename(columns = {'isDot':'dots'})

    
    df = pd.merge(innings,runs,on = 'batsman').merge( balls, on = 'batsman').merge( dismissals, on = 'batsman').merge( fours, on = 'batsman').merge( sixes, on = 'batsman').merge( dots, on = 'batsman')
    df['RPI'] = df.apply(lambda x: x['runs']/x['innings'], axis = 1)
    df['SR'] = df.apply(lambda x: 100*(x['runs']/x['balls']),axis = 1)
    df['BPD'] = df.apply(lambda x: balls_per_dismissal(x['balls'], x['dismissals']), axis = 1)
    df['BPB'] = df.apply(lambda x: balls_per_boundary(x['balls'], (x['fours'] + x['sixes'])), axis = 1)

    df['dot_percentage'] = df.apply(lambda x: x['dots']/x['balls'], axis = 1)

    return df

In [44]:
def get_phase(over_no):
    if over_no < 6:
        return'PowerPlay'
        
    elif over_no < 15:
        return'Middle Over'
    else:
        return'Death Over'

In [46]:
comb['phase'] = comb['over'].apply(lambda x: get_phase(x))

In [52]:
venues = ["Eden Gardens, Kolkata","Eden Gardens"]
teams = ["Kolkata Knight Riders"]

#print(df_result.sort_values(by='sixes', ascending=False).head())


In [56]:
df_result = ByCustom(comb, venues,"PowerPlay", teams)
df_result.sort_values(by='runs', ascending=False).head()

,batsman,innings,runs,balls,dismissals,fours,sixes,dots,RPI,SR,BPD,BPB,dot_percentage
97,S Dhawan,11,194,154,3,28,5,71,17.636364,125.974026,51.333333,4.666667,0.461039
16,CH Gayle,8,172,130,2,21,10,72,21.500000,132.307692,65.000000,4.193548,0.553846
15,BB McCullum,6,147,94,1,17,9,45,24.500000,156.382979,94.000000,3.615385,0.478723
120,V Kohli,10,130,116,6,12,4,54,13.000000,112.068966,19.333333,7.250000,0.465517
91,RG Sharma,7,118,89,1,15,6,47,16.857143,132.584270,89.000000,4.238095,0.528090


In [18]:
df_result = ByCustom(comb, venues,'Middle Over', teams)
df_result.sort_values(by='runs', ascending=False).head()

,batsman,innings,runs,balls,dismissals,fours,sixes,dots,RPI,SR,BPD,BPB,dot_percentage
13,RG Sharma,7,108,97,3,14,0,34,15.428571,111.340206,32.333333,6.928571,0.350515
14,SA Yadav,2,73,47,1,9,2,13,36.500000,155.319149,47.000000,4.272727,0.276596
15,SR Tendulkar,3,50,41,4,5,1,14,16.666667,121.951220,10.250000,6.833333,0.341463
7,KA Pollard,4,44,52,0,4,1,29,11.000000,84.615385,52.000000,10.400000,0.557692
2,DR Smith,1,32,30,1,5,0,14,32.000000,106.666667,30.000000,6.000000,0.466667


In [19]:
df_result = ByCustom(comb, venues,'Death Over', teams)
df_result.sort_values(by='innings', ascending=False).head()

,batsman,innings,runs,balls,dismissals,fours,sixes,dots,RPI,SR,BPD,BPB,dot_percentage
6,KA Pollard,6,72,39,4,2,7,12,12.000000,184.615385,9.75,4.333333,0.307692
11,RG Sharma,4,35,25,2,4,1,9,8.750000,140.000000,12.50,5.000000,0.360000
2,HH Pandya,3,102,48,0,14,4,10,34.000000,212.500000,48.00,2.666667,0.208333
3,Harbhajan Singh,3,13,8,1,1,1,3,4.333333,162.500000,8.00,4.000000,0.375000
0,AT Rayudu,1,0,1,1,0,0,1,0.000000,0.000000,1.00,1.000000,1.000000


In [20]:
comb['venue'].unique()

array(['M Chinnaswamy Stadium',
       'Punjab Cricket Association Stadium, Mohali', 'Feroz Shah Kotla',
       'Wankhede Stadium', 'Eden Gardens', 'Sawai Mansingh Stadium',
       'Rajiv Gandhi International Stadium, Uppal',
       'MA Chidambaram Stadium, Chepauk', 'Dr DY Patil Sports Academy',
       'Newlands', "St George's Park", 'Kingsmead', 'SuperSport Park',
       'Buffalo Park', 'New Wanderers Stadium', 'De Beers Diamond Oval',
       'OUTsurance Oval', 'Brabourne Stadium',
       'Sardar Patel Stadium, Motera', 'Barabati Stadium',
       'Brabourne Stadium, Mumbai',
       'Vidarbha Cricket Association Stadium, Jamtha',
       'Himachal Pradesh Cricket Association Stadium', 'Nehru Stadium',
       'Holkar Cricket Stadium',
       'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'Subrata Roy Sahara Stadium',
       'Maharashtra Cricket Association Stadium',
       'Shaheed Veer Narayan Singh International Stadium',
       'JSCA International Stadium Complex', 'Sh

In [21]:
comb['team1'].unique()

array(['Royal Challengers Bangalore', 'Kings XI Punjab',
       'Delhi Daredevils', 'Mumbai Indians', 'Kolkata Knight Riders',
       'Rajasthan Royals', 'Deccan Chargers', 'Chennai Super Kings',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Gujarat Lions', 'Rising Pune Supergiants',
       'Rising Pune Supergiant', 'Delhi Capitals', 'Punjab Kings',
       'Lucknow Super Giants', 'Gujarat Titans',
       'Royal Challengers Bengaluru'], dtype=object)

##**Head to Head In Venue Analysis**

In [162]:
from fuzzywuzzy import process
import numpy as np

def balls_per_dismissal(balls, dismissals):
    return balls / dismissals if dismissals > 0 else balls

def balls_per_boundary(balls, boundaries):
    return balls / boundaries if boundaries > 0 else balls

def ByCustom(df, current_venue=None, current_phase=None, current_opposition=None, batsmen_list=None, venue_aliases=None):
    # Ensure inputs are lists
    if current_venue and not isinstance(current_venue, list):
        current_venue = [current_venue]
    if current_opposition and not isinstance(current_opposition, list):
        current_opposition = [current_opposition]
    if venue_aliases is None:
        venue_aliases = {}  # Default empty dict for aliases

    # Convert venue aliases to a list of valid venue names
    if current_venue:
        expanded_venues = set()
        for venue in current_venue:
            expanded_venues.add(venue)  # Add original venue
            if venue in venue_aliases:
                expanded_venues.update(venue_aliases[venue])  # Add aliases

        df = df[df['venue'].isin(expanded_venues)]  # Filter by venue

    # Apply filters
    if current_phase:
        df = df[df['phase'] == current_phase]
    if current_opposition:
        df = df[df['bowling_team'].isin(current_opposition)]

    # Apply fuzzy matching for batsmen
    if batsmen_list:
        all_batsmen = df['batsman'].unique()  # Get unique batsmen names from dataset
        matched_batsmen = []
        
        for player in batsmen_list:
            best_match, score = process.extractOne(player, all_batsmen)
            if score > 80:  # Only consider matches with similarity > 80%
                matched_batsmen.append(best_match)

        df = df[df['batsman'].isin(matched_batsmen)]  # Filter only matched batsmen
    
    # Additional calculations
    df['isDot'] = df['runs_of_bat'].apply(lambda x: 1 if x == 0 else 0)
    df['isFour'] = df['runs_of_bat'].apply(lambda x: 1 if x == 4 else 0)
    df['isSix'] = df['runs_of_bat'].apply(lambda x: 1 if x == 6 else 0)

    # ✅ Runs per innings by inning type
    runs = df.groupby('batsman')['runs_of_bat'].sum().reset_index().rename(columns={'runs_of_bat': 'runs'})
    innings = df.groupby('batsman')['match_id'].nunique().reset_index().rename(columns={'match_id': 'innings'})
    balls = df.groupby('batsman').size().reset_index().rename(columns={0: 'balls'})
    dismissals = df.groupby('batsman')['player_dismissed'].count().reset_index().rename(columns={'player_dismissed': 'dismissals'})
    fours = df.groupby('batsman')['isFour'].sum().reset_index().rename(columns={'isFour': 'fours'})
    sixes = df.groupby('batsman')['isSix'].sum().reset_index().rename(columns={'isSix': 'sixes'})
    dots = df.groupby('batsman')['isDot'].sum().reset_index().rename(columns={'isDot': 'dots'})

    # ✅ Runs per innings for 1st and 2nd innings
    runs_1st = df[df['inning'] == 1].groupby('batsman')['runs_of_bat'].sum().reset_index().rename(columns={'runs_of_bat': 'runs_1st'})
    runs_2nd = df[df['inning'] == 2].groupby('batsman')['runs_of_bat'].sum().reset_index().rename(columns={'runs_of_bat': 'runs_2nd'})
    
    dismissals_1st = df[df['inning'] == 1].groupby('batsman')['player_dismissed'].count().reset_index().rename(columns={'player_dismissed': 'dismissals_1st'})
    dismissals_2nd = df[df['inning'] == 2].groupby('batsman')['player_dismissed'].count().reset_index().rename(columns={'player_dismissed': 'dismissals_2nd'})

    # ✅ Merge all data
    df = (runs.merge(innings, on='batsman')
              .merge(balls, on='batsman')
              .merge(dismissals, on='batsman')
              .merge(fours, on='batsman')
              .merge(sixes, on='batsman')
              .merge(dots, on='batsman')
              .merge(runs_1st, on='batsman', how='left').fillna(0)
              .merge(runs_2nd, on='batsman', how='left').fillna(0)
              .merge(dismissals_1st, on='batsman', how='left').fillna(0)
              .merge(dismissals_2nd, on='batsman', how='left').fillna(0))

    # ✅ Fixing Batting Average (Replace NaN with Total Runs)
    df['AVG'] = df.apply(lambda x: x['runs'] / x['dismissals'] if x['dismissals'] > 0 else x['runs'], axis=1)
    df['AVG_1'] = df.apply(lambda x: x['runs_1st'] / x['dismissals_1st'] if x['dismissals_1st'] > 0 else x['runs_1st'], axis=1)
    df['AVG_2'] = df.apply(lambda x: x['runs_2nd'] / x['dismissals_2nd'] if x['dismissals_2nd'] > 0 else x['runs_2nd'], axis=1)

    df['SR'] = (df['runs'] / df['balls']) * 100
    df['BPD'] = df.apply(lambda x: balls_per_dismissal(x['balls'], x['dismissals']), axis=1)
    df['BPB'] = df.apply(lambda x: balls_per_boundary(x['balls'], x['fours'] + x['sixes']), axis=1)
    df['dot_percentage'] = df['dots'] / df['balls']

    return df.sort_values(by='AVG', ascending=False)



In [164]:
batsmen_list = ["V Kohli", "TH David", "RM Patidar", "LS Livingstone",
                "Devdutt Padikkal", "KH Pandya", "PD Salt", "JM Sharma",
                "Romario Shepherd","Andre Russell"]

# Define venue aliases
venue_aliases = {
    "Eden Gardens, Kolkata": ["Eden Gardens"],
    "Wankhede Stadium, Mumbai": ["Wankhede Stadium"],
    "M. Chinnaswamy Stadium, Bengaluru": ["Chinnaswamy Stadium"]
}

df_filtered = ByCustom(comb, 
                       current_venue=["Eden Gardens, Kolkata"], 
                       batsmen_list=batsmen_list, 
                       venue_aliases=venue_aliases)

df_filtered.head(10)

,batsman,runs,innings,balls,dismissals,fours,sixes,dots,runs_1st,runs_2nd,dismissals_1st,dismissals_2nd,AVG,AVG_1,AVG_2,SR,BPD,BPB,dot_percentage
6,RM Patidar,164,2,79,1,15,12,23,112.0,52.0,0.0,1.0,164.000000,112.000000,52.000000,207.594937,79.000000,2.925926,0.291139
5,PD Salt,350,7,197,7,38,21,76,193.0,157.0,6.0,1.0,50.000000,32.166667,157.000000,177.664975,28.142857,3.338983,0.385787
8,V Kohli,371,12,288,10,30,13,103,158.0,213.0,4.0,6.0,37.100000,39.500000,35.500000,128.819444,28.800000,6.697674,0.357639
0,AD Russell,956,36,541,27,66,83,215,644.0,312.0,18.0,9.0,35.407407,35.777778,34.666667,176.709797,20.037037,3.630872,0.397412
1,D Padikkal,28,1,21,1,2,2,9,28.0,0.0,1.0,0.0,28.000000,28.000000,0.000000,133.333333,21.000000,5.250000,0.428571
2,JM Sharma,21,1,19,1,0,2,8,21.0,0.0,1.0,0.0,21.000000,21.000000,0.000000,110.526316,19.000000,9.500000,0.421053
3,KH Pandya,48,6,38,3,0,4,14,24.0,24.0,1.0,2.0,16.000000,24.000000,12.000000,126.315789,12.666667,9.500000,0.368421
4,LS Livingstone,15,1,9,1,3,0,3,15.0,0.0,1.0,0.0,15.000000,15.000000,0.000000,166.666667,9.000000,3.000000,0.333333
7,TH David,0,1,3,1,0,0,3,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,3.000000,3.000000,1.000000


In [158]:
from fuzzywuzzy import process
import numpy as np

def balls_per_dismissal(balls, dismissals):
    return balls / dismissals if dismissals > 0 else balls

def balls_per_boundary(balls, boundaries):
    return balls / boundaries if boundaries > 0 else balls

def ByCustom(df, current_venue=None, current_phase=None, current_opposition=None, batsmen_list=None, venue_aliases=None):
    # Ensure inputs are lists
    if current_venue and not isinstance(current_venue, list):
        current_venue = [current_venue]
    if current_opposition and not isinstance(current_opposition, list):
        current_opposition = [current_opposition]
    if venue_aliases is None:
        venue_aliases = {}  # Default empty dict for aliases

    # Convert venue aliases to a list of valid venue names
    if current_venue:
        expanded_venues = set()
        for venue in current_venue:
            expanded_venues.add(venue)  # Add original venue
            if venue in venue_aliases:
                expanded_venues.update(venue_aliases[venue])  # Add aliases

        df = df[df['venue'].isin(expanded_venues)]  # Filter by venue

    # Apply filters
    if current_phase:
        df = df[df['phase'] == current_phase]
    if current_opposition:
        df = df[df['bowling_team'].isin(current_opposition)]

    # Apply fuzzy matching for batsmen
    if batsmen_list:
        all_batsmen = df['batsman'].unique()  # Get unique batsmen names from dataset
        matched_batsmen = []
        
        for player in batsmen_list:
            best_match, score = process.extractOne(player, all_batsmen)
            if score > 80:  # Only consider matches with similarity > 80%
                matched_batsmen.append(best_match)

        df = df[df['batsman'].isin(matched_batsmen)]  # Filter only matched batsmen
    
    # Additional calculations
    df['isDot'] = df['runs_of_bat'].apply(lambda x: 1 if x == 0 else 0)
    df['isFour'] = df['runs_of_bat'].apply(lambda x: 1 if x == 4 else 0)
    df['isSix'] = df['runs_of_bat'].apply(lambda x: 1 if x == 6 else 0)

    runs = df.groupby('batsman')['runs_of_bat'].sum().reset_index().rename(columns={'runs_of_bat': 'runs'})
    innings = df.groupby('batsman')['match_id'].nunique().reset_index().rename(columns={'match_id': 'innings'})
    balls = df.groupby('batsman').size().reset_index().rename(columns={0: 'balls'})
    dismissals = df.groupby('batsman')['player_dismissed'].count().reset_index().rename(columns={'player_dismissed': 'dismissals'})
    fours = df.groupby('batsman')['isFour'].sum().reset_index().rename(columns={'isFour': 'fours'})
    sixes = df.groupby('batsman')['isSix'].sum().reset_index().rename(columns={'isSix': 'sixes'})
    dots = df.groupby('batsman')['isDot'].sum().reset_index().rename(columns={'isDot': 'dots'})
    
    # New Calculations
    highest_score = df.groupby(['batsman', 'match_id'])['runs_of_bat'].sum().reset_index().groupby('batsman')['runs_of_bat'].max().reset_index().rename(columns={'runs_of_bat': 'HS'})
    not_outs = innings.copy()
    not_outs['NotOuts'] = not_outs['innings'] - dismissals['dismissals']

    df = (runs.merge(innings, on='batsman')
              .merge(balls, on='batsman')
              .merge(dismissals, on='batsman')
              .merge(fours, on='batsman')
              .merge(sixes, on='batsman')
              .merge(dots, on='batsman')
              .merge(highest_score, on='batsman')
              .merge(not_outs[['batsman', 'NotOuts']], on='batsman'))

    # Calculate additional metrics
    df['AVG'] = df.apply(lambda x: x['runs'] / x['dismissals'] if x['dismissals'] > 0 else x['runs'], axis=1)
    df['SR'] = (df['runs'] / df['balls']) * 100
    df['BPD'] = df.apply(lambda x: balls_per_dismissal(x['balls'], x['dismissals']), axis=1)
    df['BPB'] = df.apply(lambda x: balls_per_boundary(x['balls'], x['fours'] + x['sixes']), axis=1)
    df['dot_percentage'] = df['dots'] / df['balls']

    return df.sort_values(by='AVG', ascending=False)


In [160]:
batsmen_list = ["V Kohli", "TH David", "RM Patidar", "LS Livingstone",
                "Devdutt Padikkal", "KH Pandya", "PD Salt", "JM Sharma",
                "Romario Shepherd","Andre Russell"]

# Define venue aliases
venue_aliases = {
    "Eden Gardens, Kolkata": ["Eden Gardens"],
    "Wankhede Stadium, Mumbai": ["Wankhede Stadium"],
    "M. Chinnaswamy Stadium, Bengaluru": ["Chinnaswamy Stadium"]
}

df_filtered = ByCustom(comb, 
                       current_venue=["Eden Gardens, Kolkata"], 
                       batsmen_list=batsmen_list, 
                       venue_aliases=venue_aliases)

df_filtered.head(10)


,batsman,runs,innings,balls,dismissals,fours,sixes,dots,HS,NotOuts,AVG,SR,BPD,BPB,dot_percentage
6,RM Patidar,164,2,79,1,15,12,23,112,1,164.000000,207.594937,79.000000,2.925926,0.291139
5,PD Salt,350,7,197,7,38,21,76,89,0,50.000000,177.664975,28.142857,3.338983,0.385787
8,V Kohli,371,12,288,10,30,13,103,100,2,37.100000,128.819444,28.800000,6.697674,0.357639
0,AD Russell,956,36,541,27,66,83,215,80,9,35.407407,176.709797,20.037037,3.630872,0.397412
1,D Padikkal,28,1,21,1,2,2,9,28,0,28.000000,133.333333,21.000000,5.250000,0.428571
2,JM Sharma,21,1,19,1,0,2,8,21,0,21.000000,110.526316,19.000000,9.500000,0.421053
3,KH Pandya,48,6,38,3,0,4,14,24,3,16.000000,126.315789,12.666667,9.500000,0.368421
4,LS Livingstone,15,1,9,1,3,0,3,15,0,15.000000,166.666667,9.000000,3.000000,0.333333
7,TH David,0,1,3,1,0,0,3,0,0,0.000000,0.000000,3.000000,3.000000,1.000000


#**Bowling Head to Head**

In [94]:
print(comb.columns)


Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batsman', 'bowler', 'non_striker', 'runs_of_bat', 'extra_runs',
       'total_runs', 'extras_type', 'is_wicket', 'player_dismissed',
       'dismissal_kind', 'fielder', 'season', 'city', 'date', 'match_type',
       'player_of_match', 'venue', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'winner', 'result', 'result_margin', 'target_runs',
       'target_overs', 'super_over', 'method', 'umpire1', 'umpire2', 'phase'],
      dtype='object')


In [140]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process

# ✅ Function to check if a wicket was taken by a bowler
def is_bowler_wicket(player_dismissed, dismissal_kind):
    if isinstance(player_dismissed, str):  # If a player is dismissed
        if dismissal_kind not in ['run out', 'obstructing the field', 'retired hurt']:
            return 1  # Valid bowler's wicket
    return 0

# ✅ Main Function: Filters by venue, phase, opposition & calculates bowling stats
def ByCustomBowling(comb, current_venue=None, current_phase=None, current_opposition=None, bowlers_list=None, venue_aliases=None):
    df = comb.copy()  # Work on a copy to avoid modifying original data

    # 🔹 Apply Venue Filtering
    if current_venue:
        if isinstance(current_venue, str):
            current_venue = [current_venue]

        # Expand venue aliases if provided
        if venue_aliases:
            expanded_venues = set(current_venue)
            for venue in current_venue:
                if venue in venue_aliases:
                    expanded_venues.update(venue_aliases[venue])

            df = df[df['venue'].isin(expanded_venues)]
        else:
            df = df[df['venue'].isin(current_venue)]

    # 🔹 Apply Phase Filtering
    if current_phase and 'phase' in df.columns:
        df = df[df['phase'] == current_phase]

    # 🔹 Apply Opposition Filtering
    if current_opposition:
        df = df[df['batting_team'].isin(current_opposition)]

    # 🔹 Apply Fuzzy Matching for Bowlers
    if bowlers_list:
        all_bowlers = df['bowler'].unique()
        matched_bowlers = []
        for player in bowlers_list:
            best_match, score = process.extractOne(player, all_bowlers)
            if score > 80:  # Consider matches with similarity > 80%
                matched_bowlers.append(best_match)
        df = df[df['bowler'].isin(matched_bowlers)]

    # 🔹 Count wickets for bowler (excluding run-outs)
    df['isBowlerWk'] = df.apply(lambda x: is_bowler_wicket(x['player_dismissed'], x['dismissal_kind']), axis=1)

    # 🔹 Count valid balls (ignores wides)
    df['valid_ball'] = df['extra_runs'].apply(lambda x: 0 if x == 'wides' else 1)

    # 🔹 Calculate dot balls
    df['isDot'] = df.apply(lambda x: 1 if x['runs_of_bat'] == 0 and x['valid_ball'] == 1 else 0, axis=1)

    # 🔹 Count wickets separately for 1st & 2nd innings
    df['wickets_1st_innings'] = df.apply(lambda x: x['isBowlerWk'] if x['inning'] == 1 else 0, axis=1)
    df['wickets_2nd_innings'] = df.apply(lambda x: x['isBowlerWk'] if x['inning'] == 2 else 0, axis=1)

    # 🔹 Group by bowler to calculate stats
    bowling_stats = df.groupby('bowler').agg(
        balls_bowled=('valid_ball', 'sum'),  
        runs_given=('total_runs', 'sum'),  
        wickets=('isBowlerWk', 'sum'),  
        dot_balls=('isDot', 'sum'),
        wickets_1st_innings=('wickets_1st_innings', 'sum'),
        wickets_2nd_innings=('wickets_2nd_innings', 'sum')
    ).reset_index()

    # 🔹 Calculate additional metrics
    bowling_stats['total_wickets'] = bowling_stats['wickets_1st_innings'] + bowling_stats['wickets_2nd_innings']
    bowling_stats['Economy'] = bowling_stats['runs_given'] / (bowling_stats['balls_bowled'] / 6)
    bowling_stats['Strike Rate'] = bowling_stats.apply(lambda x: x['balls_bowled'] / x['wickets'] if x['wickets'] > 0 else np.nan, axis=1)
    bowling_stats['Bowling Average'] = bowling_stats.apply(lambda x: x['runs_given'] / x['wickets'] if x['wickets'] > 0 else np.nan, axis=1)
    bowling_stats['Dot %'] = (bowling_stats['dot_balls'] / bowling_stats['balls_bowled']) * 100
    bowling_stats['BPD'] = bowling_stats.apply(lambda x: x['balls_bowled'] / x['wickets'] if x['wickets'] > 0 else np.nan, axis=1)  # ✅ Balls Per Dismissal

    return bowling_stats.sort_values(by='total_wickets', ascending=False)

# ✅ Example usage
bowlers_list = ["CV Varun", "SP Narine", "Harshit Rana", "Vaibhav Arora", "Moeen Ali",
                "Chetan Sakariya", "Andre Russell", "Anukul Roy", "Anrich Nortje",
                "Mayank Markande", "Spencer Johnson", "Venkatesh Iyer"]

venue_aliases = {
    "Eden Gardens, Kolkata": ["Eden Gardens"],
    "Wankhede Stadium, Mumbai": ["Wankhede Stadium"],
    "M. Chinnaswamy Stadium, Bengaluru": ["Chinnaswamy Stadium"]
}

df_bowling = ByCustomBowling(comb, 
                             current_venue="Eden Gardens, Kolkata", 
                             current_phase=None, 
                             current_opposition=None, 
                             bowlers_list=bowlers_list, 
                             venue_aliases=venue_aliases)

df_bowling


,bowler,balls_bowled,runs_given,wickets,dot_balls,wickets_1st_innings,wickets_2nd_innings,total_wickets,Economy,Strike Rate,Bowling Average,Dot %,BPD
7,SP Narine,1342,1426,70,575,29,41,70,6.375559,19.171429,20.371429,42.846498,19.171429
0,AD Russell,627,951,39,248,16,23,39,9.100478,16.076923,24.384615,39.553429,16.076923
2,CV Varun,347,521,21,144,11,10,21,9.008646,16.523810,24.809524,41.498559,16.523810
3,Harshit Rana,239,376,15,101,5,10,15,9.439331,15.933333,25.066667,42.259414,15.933333
5,MG Johnson,191,242,9,98,5,4,9,7.602094,21.222222,26.888889,51.308901,21.222222
8,VG Arora,127,192,6,62,5,1,6,9.070866,21.166667,32.000000,48.818898,21.166667
4,M Markande,72,93,5,30,2,3,5,7.750000,14.400000,18.600000,41.666667,14.400000
6,MM Ali,12,33,1,4,0,1,1,16.500000,12.000000,33.000000,33.333333,12.000000
1,AS Roy,18,56,0,3,0,0,0,18.666667,NaN,NaN,16.666667,NaN
